In [28]:
import copy
from fireplace.exceptions import GameOver
from fireplace.utils import play_full_game
import logging
import random
from fireplace import cards

%load_ext snakeviz

cards.db.initialize()
logging.disable(logging.WARNING) #logging.NOTSET

The snakeviz extension is already loaded. To reload it, use:
  %reload_ext snakeviz


In [22]:
def run_game():
    try:
        play_full_game();
    except GameOver:
        pass

In [17]:
%%capture c
random.seed(1857)
%timeit -n100 run_game()

In [18]:
c.stdout.splitlines()[-1]

'100 loops, best of 3: 130 ms per loop'

In [30]:
%%snakeviz
%%capture
random.seed(1857)
for _ in range(100):
    run_game()



 
*** Profile stats marshalled to file '/var/folders/hm/t9c16vnj7_sbskcdnsqb7wh4003rpx/T/tmph2on16hg'. 
